In [2]:
import os
import time
import pandas as pd

# pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup # pip install beautifulsoup4



## FUNCIONES

In [3]:
"--------------  ENTRAMOS EN CONSUM ---------------------"

def enter_consum():

    # Navigate to the main page
    link = f'https://tienda.consum.es/es#!Home'
    driver.get(link)

    #Reject Cookies :
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'RECHAZAR TODAS')]"))).click()
    


In [4]:
def expand_cat_menu():
    driver.find_element(By.CLASS_NAME, "menu-button").click()
    return

In [5]:
def category_subcategory():
   
    #Find navigation panel 
    left_panel = driver.find_element(By.ID, "panel-component")

    #The main navigation panel (left_panel) have tree sections: 
    sections = left_panel.find_elements(By.CLASS_NAME, "element-list")

    #Acces section that contains Categories (last one)
    section_categories = sections[-1]
    
    # Extracts a list with the name of all sections: 
    category_names = section_categories.text.split("\n")
    category_names.remove("Ecológico y saludable")
    
    #create an empty dictionary:
    
    cat_subcat_dict = {}
    
    for c in category_names[:-1]:
        
        "When you place the mouse in a category name, the rigth sub-category panel changes"

        # Targets each category_element()
        categorie_element = section_categories.find_element(By.LINK_TEXT, f"{c}")
        time.sleep(1)
        actions.move_to_element(categorie_element).perform() #Sub-category panel changes
        
        # Targets sub-panel that contain sub cateogies name:
        sub_panel  = driver.find_element(By.XPATH, "//cmp-panel[contains(@class, 'left-panel__subcategories')]")
        
        #extract a list with the name of the sub-categories
        sub_category_names = sub_panel.text.split("\n")
        
        cat_subcat_dict[c] = sub_category_names
        
    driver.find_element(By.CSS_SELECTOR, "cmp-icon[name='icon-cancel']").click()
    
    return cat_subcat_dict

In [6]:

def navigate(c):

    expand_cat_menu()

    #Find navigation panel 
    
    left_panel = driver.find_element(By.ID, "panel-component")

    #The main navigation panel (left_panel) have tree sections: 
    sections = left_panel.find_elements(By.CLASS_NAME, "element-list")

    #Acces section that contains Categories (last one)
    section_categories = sections[-1]
    
    wait.until(EC.visibility_of_element_located((By.LINK_TEXT, f"{c}")))
    
    categorie_element = section_categories.find_element(By.LINK_TEXT, f"{c}")

    time.sleep(1)

    actions.move_to_element(categorie_element).perform()
    
    return


In [7]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}_10-03.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

## MAIN

In [8]:
df = pd.DataFrame(columns=['categoria', 'subcategoria', 'descripcion', "unidades", "precio", "marca"])
df

,categoria,subcategoria,descripcion,unidades,precio,marca


In [16]:
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 20)

driver.maximize_window()

#Enter de supermarket shop place:
enter_consum()

expand_cat_menu() #Expande menu categories

#Extract categories y subcategories(dict)
os.system("say Creando diccionario categorias")

cat_subcat_dict = category_subcategory()
#driver.find_element(By.CSS_SELECTOR, "cmp-icon[name='icon-cancel']").click()

os.system("Diccionario Creado")


# Itereate for each category and subcategory and acces the page
for keys, values in cat_subcat_dict.items():
    os.system(f"say Entrando en  {keys}")
    for val in values:
        os.system(f"say Entrando en {val}")
        #Function that acces a given category
        navigate(keys)
        # Locate the selected subcategory inside de category - click - loads page
        wait.until(EC.visibility_of_element_located((By.LINK_TEXT, f"{val}"))).click()
        
        time.sleep(5)
        
        # Nº of products in this subcategory:
        total_elements = driver.find_element(By.ID, "grid-totalproducts").text.replace(" productos", "")

        if int(total_elements) > 20:
            # Nº of pages in witch the products are display: 
            total_paginas = driver.find_element(By.ID, "paginator-dropdown").text.split("\n")[-1].replace("de ", "")   
        else:
            total_paginas = 1
            
        for i in range(int(total_paginas)):
            
            os.system(f"say pagina {i+1} de {total_paginas}")
            
            time.sleep(5)
            
            wait.until(EC.visibility_of_element_located((By.XPATH, "//cmp-widget-product[contains(@class, 'grid__widget--prod')]")))
            products = driver.find_elements(By.XPATH, "//cmp-widget-product[contains(@class, 'grid__widget--prod')]")
            
            if len(products) == 20

            time.sleep(3)

            
            for p in products:
                
                row = {
                    "categoria":keys , 
                    "subcategoria": val, 
                    "descripcion": p.find_element(By.XPATH, "//h3[contains(@class, 'widget-prod__info-description')]").text, 
                    "unidades": p.find_element(By.XPATH, "//div[contains(@class, 'widget-prod__info-unitprice')]").text, 
                    "precio": p.find_element(By.XPATH, "//div[contains(@class, 'widget-prod__price')]").text,
                    "marca": p.find_element(By.XPATH, "//div[contains(@class, 'widget-prod__info-brand')]").text
                    }

                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                    
                print(f"he añadido{descripcion}")

                    #print(f"exito en {keys} --- {val}")

            if i == range(int(total_paginas))[-1]:
                break
            else:
                next_page = driver.find_element(By.CLASS_NAME, "next-page")
                actions.move_to_element(next_page).perform()
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "next-page"))).click()
                time.sleep(1)
                
            
        #status = f"{keys} -- {val} -- {total_paginas} pagina/s {total_elements} elementos"
        #print(status)
            
os.system("Consum ha sido hackeada con éxito")
export_to_csv(df,"consum")

sh: Diccionario: command not found


producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido
producto no añadido


KeyboardInterrupt: 

In [40]:
driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 20)

driver.maximize_window()

#Enter de supermarket shop place:
enter_consum()

In [41]:
products = driver.find_elements(By.XPATH, "//cmp-widget-product[contains(@class, 'grid__widget--prod')]")

In [42]:
products[5].text

'SALADITOS\nAltramuces Tarro 380 Gr\n4,08 € / 1 Kg\n1,69 €\n1,55 €\nAñadir'

In [47]:
p = products[5]

p.find_element(By.CSS_SELECTOR, "[class*= 'widget-prod__info-description']").text

#p.find_element(By.XPATH, "h3[contains(@class, 'widget-prod__info-description')]").text, 

'Altramuces Tarro 380 Gr'